In [ ]:
!pip install dash sqlalchemy pymysql pandas plotly



In [3]:
import pandas as pd
from sqlalchemy import create_engine

#DB credential
db_user='root'
db_password='9728686499'
db_host='localhost'
db_name='airport_db'


#create connection engine
engine=create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')


#Query the data
query="select * from airport_short"
df=pd.read_sql(query,engine)
df.head()


,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NA,NA\r
1,SEA,RDM,"Seattle, WA","Bend, OR",126,148,4,228,2008-11-01,6713274,157730,47.449001,-122.308998,44.2541008,-121.1500015\r
2,SEA,RDM,"Seattle, WA","Bend, OR",61,70,1,228,2008-11-01,6713274,157730,47.449001,-122.308998,44.2541008,-121.1500015\r
3,SEA,RDM,"Seattle, WA","Bend, OR",666,1073,29,228,2008-02-01,6713274,157730,47.449001,-122.308998,44.2541008,-121.1500015\r
4,GEG,RDM,"Spokane, WA","Bend, OR",13,30,1,291,2008-02-01,462408,157730,47.619900,-117.533997,44.2541008,-121.1500015\r


In [9]:
##create simple dash application
from dash import Dash,dcc,html
import plotly.express as px


##create Dash app
app=Dash(__name__)


##lets plot using plotly (Passengers data over Distance)
fig=px.scatter(df,x='Distance',y='Passengers',color='Origin_airport',title='Passengers vs distance',
               labels={'Passengers':'Passengers','Distance':'Distance(Miles)'})

fig_bar=px.bar(df,x='Origin_airport',y='Seats',color='Destination_airport',title='Seats per flight',labels={'Seats':'No. of Seats','Origin_airport':'Origin_airport'})

app.layout=html.Div(children=[
    html.H1(children='Flight Dashboard'),
    dcc.Graph(
        id='Passneger-distance-graph',
        figure=fig
    ),
    dcc.Graph(
        id='Seats-bar-chart',
        figure=fig_bar
    ),
    html.Div(children='Dashboard for visualization of flight data')
])

if __name__=='__main__':
    app.run(debug=True)
                   



In [24]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Dummy dataframe (replace this with your actual df from SQL)
# df = pd.read_sql(query, engine)

# For now, assuming df is already loaded
# Make sure df['Fly_date'] is datetime
df['Fly_date'] = pd.to_datetime(df['Fly_date'])

# Create the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Flight Dashboard'),

    dcc.Dropdown(
        id='Origin_airport dropdown',
        options=[{'label': airport, 'value': airport} for airport in df['Origin_airport'].unique()],
        placeholder='Select an origin airport',
        multi=True
    ),

    dcc.DatePickerRange(
        id='Date-Picker-Range',
        start_date=df['Fly_date'].min(),
        end_date=df['Fly_date'].max(),
        display_format='YYYY-MM-DD'
    ),

    dcc.Graph(id='Filtered-Passenger-Distance-graph'),
    dcc.Graph(id='Filtered-Seats-bar-chart')
])

# Callback
@app.callback(
    [
        Output('Filtered-Passenger-Distance-graph', 'figure'),
        Output('Filtered-Seats-bar-chart', 'figure')
    ],
    [
        Input('Origin_airport dropdown', 'value'),
        Input('Date-Picker-Range', 'start_date'),
        Input('Date-Picker-Range', 'end_date')
    ]
)
def update_graphs(selected_airports, start_date, end_date):
    # ✅ Make sure this line is inside the function
    filtered_df = df[(df['Fly_date'] >= start_date) & (df['Fly_date'] <= end_date)]

    if selected_airports:
        filtered_df = filtered_df[filtered_df['Origin_airport'].isin(selected_airports)]

    scatter_fig = px.scatter(
        filtered_df,
        x='Distance',
        y='Passengers',
        color='Origin_airport',
        title='Filtered Passengers vs Distance',
        labels={'Passengers': 'Passengers', 'Distance': 'Distance (Miles)'}
    )

    bar_fig = px.bar(
        filtered_df,
        x='Origin_airport',
        y='Seats',
        color='Destination_airport',
        title='Filtered Seats per Flight'
    )

    return scatter_fig, bar_fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
